# Track Lineage

To track data lineage in Azure Purview, you can use the Purview API to create or update relationships between different data entities in your catalog. Lineage typically involves mapping data flow between source and target entities, such as databases, tables, or files.


## Using API Key

In [1]:
import requests
import msal
from azure.identity import ClientSecretCredential


In [ ]:

# Azure AD credentials and Purview settings
client_id = '<YOUR_CLIENT_ID>'
client_secret = '<YOUR_CLIENT_SECRET>'
tenant_id = '<YOUR_TENANT_ID>'
purview_account_name = '<YOUR_PURVIEW_ACCOUNT_NAME>'
purview_endpoint = f'https://{purview_account_name}.purview.azure.com'

# Authenticate using ClientSecretCredential
def get_access_token():
    credential = ClientSecretCredential(tenant_id, client_id, client_secret)
    token = credential.get_token('https://purview.azure.net/.default')
    return token.token

# Example function to create lineage between two assets
def create_lineage(source_entity_guid, target_entity_guid):
    url = f"{purview_endpoint}/catalog/api/atlas/v2/lineage/relations"
    headers = {
        "Authorization": f"Bearer {get_access_token()}",
        "Content-Type": "application/json"
    }

    lineage_payload = {
        "relationshipType": "DataFlow",
        "end1": {"guid": source_entity_guid},
        "end2": {"guid": target_entity_guid}
    }

    response = requests.post(url, json=lineage_payload, headers=headers)

    if response.status_code == 200:
        print(f"Lineage created between {source_entity_guid} and {target_entity_guid}.")
    else:
        print(f"Failed to create lineage: {response.status_code}, {response.text}")

# Example usage
source_guid = "<SOURCE_ENTITY_GUID>"  # Replace with actual entity GUID
target_guid = "<TARGET_ENTITY_GUID>"  # Replace with actual entity GUID
create_lineage(source_guid, target_guid)


## Using MFA

In [ ]:


# Azure AD credentials and Purview settings
client_id = '<YOUR_CLIENT_ID>'
tenant_id = '<YOUR_TENANT_ID>'
purview_account_name = '<YOUR_PURVIEW_ACCOUNT_NAME>'
purview_endpoint = f"https://{purview_account_name}.purview.azure.com"

# Authentication authority and scope for Purview API
authority = f"https://login.microsoftonline.com/{tenant_id}"
scope = ["https://purview.azure.net/.default"]  # Purview API scope

# Initialize MSAL public client application for device code flow
app = msal.PublicClientApplication(client_id, authority=authority)

# Initiate the device code flow for MFA authentication
flow = app.initiate_device_flow(scopes=scope)
if "user_code" not in flow:
    raise Exception("Device flow initiation failed.")

# Display the code and URL for the user to authenticate
print(f"Go to {flow['verification_uri']} and enter the code: {flow['user_code']}")

# Wait for user to complete authentication and MFA
token_result = app.acquire_token_by_device_flow(flow)
if "access_token" not in token_result:
    raise Exception(f"Authentication failed: {token_result.get('error_description')}")

access_token = token_result['access_token']

# Example function to create lineage between two assets
def create_lineage(source_entity_guid, target_entity_guid):
    url = f"{purview_endpoint}/catalog/api/atlas/v2/lineage/relations"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    lineage_payload = {
        "relationshipType": "DataFlow",
        "end1": {"guid": source_entity_guid},
        "end2": {"guid": target_entity_guid}
    }

    response = requests.post(url, json=lineage_payload, headers=headers)

    if response.status_code == 200:
        print(f"Lineage created between {source_entity_guid} and {target_entity_guid}.")
    else:
        print(f"Failed to create lineage: {response.status_code}, {response.text}")

# Example usage
source_guid = "<SOURCE_ENTITY_GUID>"  # Replace with actual entity GUID
target_guid = "<TARGET_ENTITY_GUID>"  # Replace with actual entity GUID
create_lineage(source_guid, target_guid)
